In [1]:
import numpy as np

r = np.random.beta(0.1, 1)
print r

1.99104426988e-11


In [48]:
from xml.etree import cElementTree as ET
import sys

posts = open('Posts.xml', 'r').read()

posts[1:100]

def remove_tags(text):
    return ''.join(ET.fromstring(text).itertext())

root = ET.fromstring(posts)
documents = []
for child in root.findall('row')[0:200]:
    text = None
    try:
        child_text = child.get('Body').encode('utf-8').strip()
        text = remove_tags(child_text)
    except ET.ParseError as e:
        #print("Caught exception: " + str(e))
        "Caught exception: " + str(e)
    if text != None: documents.append(text)
        
    
print documents[0:10]

['I was offered a beer the other day that was reportedly made with citra hops. What are citra hops? Why should I care that my beer is made with them?', 'As far as we know, when did humans first brew beer, and where? Around when would you have been able to get your hands on something resembling a modern lager?', "How is low/no alcohol beer made? I'm assuming that the beer is made normally and the alcohol is then removed, is it any more than just boiling it off? I've noticed that no/low alcohol beers' taste improved hugely a few years ago, is this due to a new technique?", "In general, what's the best way to work out the temperature at which to serve a particular beer? Room temperature? Cold? Supercold? Warm?", "Assuming we're comparing equivalent amounts of alcohol, do certain beers get you inebriated more quickly or slowly? Does the amount of fizz make a difference?", 'Apart from coming out of different taps, some ales seem very similar to lagers (although there are clearly a much grea

In [49]:
execfile('corpus.py')
execfile('document.py')

corpus = Corpus(documents, '../Week1HW/stopwords.txt', 3)

print corpus.docs[0].tokens
# FIXME: Not sure why this is not 100
print corpus.N

[u'wa' u'offer' u'beer' u'the' u'day' u'wa' u'reportedli' u'citra' u'hop'
 u'are' u'citra' u'hop' u'whi' u'care' u'beer']
62


In [60]:
# Initizations
#
D = len(corpus.docs)
K = 3 # K - numer of topics
V = len(corpus.token_set)

# Randomly assign each word in each document to a topic
# each array is of "arbitrary" length - length of document's respective word list
word_topic_assignments = [[]]*corpus.N
for doci, doc in enumerate(corpus.docs):
    word_topic_assignments[doci] = []
    for wordi, word in enumerate(doc.tokens):
        word_topic_assignments[doci].append(np.random.randint(0,K))

#print word_topic_assignments[0]

# Initialize document counts of words for each topic, D x K
document_topic_word_distribution = np.zeros((D, K))
for doci, doc in enumerate(corpus.docs):
    # sum the number of words in topic k
    for k in range(0,K):
        document_topic_word_distribution.itemset((doci, k), word_topic_assignments[doci].count(k))

#print document_topic_word_distribution[0,:]

# Initialize term topic counts, e.g. number of times term V was allocated to topic K, K x V
# For every word in every document, determine what it's term index is
topic_term_distribution = np.zeros((K, V))
termlist = list(corpus.token_set)
for doci, doc in enumerate(corpus.docs):
    for wordi, word in enumerate(doc.tokens):
        termidx = termlist.index(word)
        topic_alloc = word_topic_assignments[doci][wordi]
        current_count = topic_term_distribution.item((topic_alloc,termidx))
        topic_term_distribution.itemset((topic_alloc,termidx), current_count + 1)
        
#print topic_term_distribution[0,:][0:10]

alpha = 50/K
eta = 200/V

# Initialize theta - document-specific topic probabilities
theta = np.zeros((D, K))
# draw theta - Dirichlet with paramters (alpha + n_{d,k}) - num of words in doc with topic alloc k
for doci in range(D):
    theta_params = alpha + document_topic_word_distribution[doci,:]
    # theta for this document
    theta_d = np.random.dirichlet(tuple(theta_params), 1)
    theta[doci,:] = theta_d

# Initialize beta - topic-specific term probabilities
beta = np.zeros((K, V))
# draw beta - Dirichlet with paramters (eta + m_{k,v}) - num of times term appeared in topic k
for k in range(K):
    beta_params = eta + topic_term_distribution[k,:]
    beta_k = np.random.dirichlet(tuple(beta_params), 1)
    beta[k,:] = beta_k
    
terms_to_sanity_check = ['skunk', 'can', 'wine', 'saison']
for term in terms_to_sanity_check:
    termidx = termlist.index(term)    
    print 'Term: ' + term
    print 'Original topic distribution: ' + str(topic_term_distribution[:,termidx])

iters = 10
for i in range(iters):
    if i%25 == 0: print 'Iteration ' + str(i)
    # 1 iteration
    for doci, doc in enumerate(corpus.docs):
        theta_d = theta[doci,:]
        if i == 0 and doci == 0: print theta_d
        for wordi, word in enumerate(doc.tokens):
            word_topic_alloc = word_topic_assignments[doci][wordi]
            termidx = termlist.index(word)
            beta_v = beta[:,termidx]
            if i == 0 and doci == 0: print beta_v            
            # decrement counts
            current_doc_count = document_topic_word_distribution.item((doci, word_topic_alloc))
            document_topic_word_distribution.itemset((doci, word_topic_alloc), current_doc_count - 1)
            current_topic_count = topic_term_distribution.item((word_topic_alloc, termidx))
            topic_term_distribution.itemset((word_topic_alloc, termidx), current_topic_count - 1)
            probs = theta_d*beta_v/np.sum(theta_d*beta_v)
            new_topic_alloc = np.random.multinomial(1, tuple(probs))
            new_topic_alloc = list(new_topic_alloc).index(1)                
            word_topic_assignments[doci][wordi] = new_topic_alloc
            # increment counts
            current_doc_count = document_topic_word_distribution.item((doci, new_topic_alloc))
            document_topic_word_distribution.itemset((doci, new_topic_alloc), current_doc_count + 1)
            current_topic_count = topic_term_distribution.item((new_topic_alloc, termidx))
            topic_term_distribution.itemset((new_topic_alloc, termidx), current_topic_count + 1) 
    # update theta
    for doci in range(D):
        theta_params = alpha + document_topic_word_distribution[doci,:]
        # theta for this document
        theta_d = np.random.dirichlet(tuple(theta_params), 1)
        theta[doci,:] = theta_d
    for k in range(K):
        beta_params = eta + topic_term_distribution[k,:]
        beta_k = np.random.dirichlet(tuple(beta_params), 1)
        beta[k,:] = beta_k    
        
for termi, term in enumerate(terms_to_sanity_check):
    termidx = termlist.index(term)
    print 'Term: ' + term
    print 'New topic distribution: ' + str(topic_term_distribution[:,termidx])


Term: skunk
Original topic distribution: [ 0.  7.  1.]
Term: can
Original topic distribution: [ 6.  3.  7.]
Term: wine
Original topic distribution: [ 0.  2.  3.]
Term: saison
Original topic distribution: [ 0.  1.  3.]
Iteration 0
[ 0.44160843  0.24262541  0.31576616]
[ 0.00241498  0.          0.00109603]
[ 0.          0.          0.00135199]
[ 0.05230083  0.0298082   0.03281211]
[ 0.05411699  0.05754022  0.07095561]
[ 0.          0.00094788  0.        ]
[ 0.00241498  0.          0.00109603]
[ 0.00110351  0.          0.        ]
[ 0.          0.          0.00252525]
[ 0.0013333   0.00246695  0.00968868]
[ 0.01253634  0.00570346  0.0400965 ]
[ 0.          0.          0.00252525]
[ 0.0013333   0.00246695  0.00968868]
[ 0.00960969  0.00358822  0.00498768]
[ 0.00132675  0.00015784  0.        ]
[ 0.05230083  0.0298082   0.03281211]
Term: skunk
New topic distribution: [ 0.  8.  0.]
Term: can
New topic distribution: [  6.   0.  10.]
Term: wine
New topic distribution: [ 0.  0.  5.]
Term: saison